# Analyze SDG Data availability

## 1. Load python libraries

In [92]:
import pandas as pd
import os
import csv
import json
import statistics

import numpy as np
from pathlib import Path

dir_path = os.path.dirname(os.path.realpath('__file__'))
print(dir_path)

# https://volderette.de/jupyter-notebook-tip-multiple-outputs/
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

C:\Users\gonzalezmorales\projects\fis4sdg_2020\notebooks\reports


## 2. Set release

In [93]:
release = '2020.Q2.G.03'

## 3. Read M49 countries and groups

In [94]:
# List or countries for which availability will be calculated 
# (currently includes 193 UN Member States plus State of Palestine)
countries = pd.read_excel('../../data/external/m49_availability.xlsx', sheet_name='Countries')

# List of regions for which availability will be calculated
regions = pd.read_excel('../../data/external/m49_availability.xlsx', sheet_name='Regions')

countries.head()
regions.head()

,countryCode,countryName,LDC,LLDC,SIDS,Development,regionCode,regionName
0,4,Afghanistan,x,x,NaN,Developing,62,Central and Southern Asia
1,8,Albania,NaN,NaN,NaN,Developed,513,Europe and Northern America
2,12,Algeria,NaN,NaN,NaN,Developing,747,Northern Africa and Western Asia
3,20,Andorra,NaN,NaN,NaN,Developed,513,Europe and Northern America
4,24,Angola,x,NaN,NaN,Developing,202,Sub-Saharan Africa


,Code,SDG_Region_Name,Unnamed: 2,Unnamed: 3
0,1.0,World,NaN,NaN
1,747.0,Northern Africa and Western Asia,NaN,NaN
2,202.0,Sub-Saharan Africa,NaN,NaN
3,419.0,Latin America and the Caribbean,NaN,NaN
4,513.0,Europe and Northern America,NaN,NaN


## 3. List of Goals, Targets, Series and Indicators

In [95]:

# The medata_[release].json file includes the list of Goals, Targets, 
# Indictors and Series available in the database, along with their 
# descriptions and other annotations

metadatafile = 'metadata_'+release+'.json'

with open('../../data/external/'+metadatafile) as json_file:
    metadata = json.load(json_file)


# Goal-specific attributes:
metadata[0].keys()

# Target-specific attributes:
metadata[0]['targets'][0].keys()

# Indicator-specific attributes
metadata[0]['targets'][0]['indicators'][0].keys()

metadata[0]['targets'][0]['indicators'][0]['series'][0]


dict_keys(['code', 'labelEN', 'descEN', 'labelES', 'descES', 'labelFR', 'descFR', 'labelRU', 'descRU', 'labelZN', 'descZN', 'hex', 'rgb', 'ColorScheme', 'thumbnail', 'targets'])

dict_keys(['code', 'labelEN', 'descEN', 'labelES', 'descES', 'labelFR', 'descFR', 'labelRU', 'descRU', 'labelZN', 'descZN', 'indicators'])

dict_keys(['code', 'reference', 'labelEN', 'descEN', 'labelES', 'descES', 'labelFR', 'descFR', 'labelRU', 'descRU', 'labelZN', 'descZN', 'series'])

{'code': 'SI_POV_DAY1',
 'description': 'Proportion of population below international poverty line (%)',
 'release': '2020.Q2.G.03',
 'tags': ['poverty line', 'poverty', 'standard of living', 'basic needs']}

## 4. Define auxiliary functions that will be used in this script

#### Read a csv file containing a series dataset

In [96]:
def read_sdg_dataset(csvfile):
    return pd.read_csv('../../data/processed/'+release+'/'+csvfile, sep='\t') 


#### Build csv file name

In [97]:
def csv_file(indicator, series):
    return ('Indicator_' + indicator.replace('.', '_') + '__Series_' + series + '.csv')

#### Identify disaggregations

Given a data set, identify columns that correpond to 'dimensions' / 'disaggregations'.  
- Exclude all Goal, Target, Indicator, and Series codes and descriptions
- Exclude all geographic codes and descriptions as well as coordinates
- Exclude annotations (e.g., "UN Member"...)
- Exclude time-reladed columns 
- Exclude attributes (units, reporting type, base period, footnotes, etc)
- Exclude 'value' columns
- Exclude upper/lower bound columns
- Exclude 'description' columns

! Note that "nature" is not included as dimension

In [98]:
def get_disaggregations(data):
   
    dim_columns = list(data.columns)
    exclude_cols = ['goal_code','goal_labelEN','goal_descEN','target_code','target_descEN',
                'indicator_code','indicator_reference','indicator_descEN',
                'series_release','series_tags','series','seriesDescription',
                'geoAreaCode','geoAreaName','level','parentCode','parentName','type',
                'X','Y','ISO3','UN_Member','Country_Profile','timeCoverage','geoInfoUrl',
                'years','min_year','max_year','n_years',
                'unitsCode','unitsDesc','reportingTypeCode','reportingTypeDesc',
                'latest_value','basePeriod','valueDetails','footnotes','sources','timeDetails','nature']

    d = [i for i in dim_columns if i not in exclude_cols and not i.startswith('value_') and not i.startswith('upper') and not i.startswith('lower') and not i.endswith('_desc')] 
    
    #return [sub.replace('_code', '') for sub in d] 
    return d


## Calculate availability, setp-by-step 

### Pick a series to run the example:

In [102]:
g = metadata[0]
t = g['targets'][0]
i = t['indicators'][0]
s = i['series'][1]

print(f'---> We will work with the following example:')
print(f"Indicator {i['reference']}, Series {s['code']}: {s['description']}.")

---> We will work with the following example:
Indicator 1.1.1, Series SI_POV_EMP1: Employed population below international poverty line, by sex and age (%).


### Read the csv file:

In [103]:
f = csv_file(i['reference'], s['code'])
                      
data = read_sdg_dataset(f)

data.head()

,goal_code,goal_labelEN,goal_descEN,target_code,target_descEN,indicator_code,indicator_reference,indicator_descEN,series_release,series_tags,...,value_2017,value_2018,value_2019,latest_value,basePeriod,valueDetails,footnotes,sources,timeDetails,nature
0,1,Goal 1,End poverty in all its forms everywhere,1.1,"By 2030, eradicate extreme poverty for all peo...",C010101,1.1.1,Proportion of the population living below the ...,2020.Q2.G.03,"['poverty line', 'poverty', 'standard of livin...",...,7.7,7.4,7.1,7.1,NaN,NaN,NaN,"ILO modelled estimates, November 2019, availab...",NaN,M: Modeled data
1,1,Goal 1,End poverty in all its forms everywhere,1.1,"By 2030, eradicate extreme poverty for all peo...",C010101,1.1.1,Proportion of the population living below the ...,2020.Q2.G.03,"['poverty line', 'poverty', 'standard of livin...",...,6.8,6.5,6.3,6.3,NaN,NaN,NaN,"ILO modelled estimates, November 2019, availab...",NaN,M: Modeled data
2,1,Goal 1,End poverty in all its forms everywhere,1.1,"By 2030, eradicate extreme poverty for all peo...",C010101,1.1.1,Proportion of the population living below the ...,2020.Q2.G.03,"['poverty line', 'poverty', 'standard of livin...",...,7.9,7.7,7.5,7.5,NaN,NaN,NaN,"ILO modelled estimates, November 2019, availab...",NaN,M: Modeled data
3,1,Goal 1,End poverty in all its forms everywhere,1.1,"By 2030, eradicate extreme poverty for all peo...",C010101,1.1.1,Proportion of the population living below the ...,2020.Q2.G.03,"['poverty line', 'poverty', 'standard of livin...",...,7.1,6.8,6.6,6.6,NaN,NaN,NaN,"ILO modelled estimates, November 2019, availab...",NaN,M: Modeled data
4,1,Goal 1,End poverty in all its forms everywhere,1.1,"By 2030, eradicate extreme poverty for all peo...",C010101,1.1.1,Proportion of the population living below the ...,2020.Q2.G.03,"['poverty line', 'poverty', 'standard of livin...",...,13.2,12.8,12.5,12.5,NaN,NaN,NaN,"ILO modelled estimates, November 2019, availab...",NaN,M: Modeled data


#### Identify columns for available disaggregations

In [105]:
disaggregations = get_disaggregations(data)

print(f"disaggregations for {s['code']}:")
disaggregations



disaggregations for SI_POV_EMP1:


['age_code', 'sex_code']

#### Columns with geography and time-period information

In [106]:
geo_and_time = ['geoAreaCode', 'geoAreaName','years', 'min_year', 'max_year', 'n_years']
print(f"columns with geography and time-period information for {s['code']}:")
geo_and_time


columns with geography and time-period information for SI_POV_EMP1:


['geoAreaCode', 'geoAreaName', 'years', 'min_year', 'max_year', 'n_years']

#### Join country information and series dataset:

In [107]:
country_data = pd.merge(countries, 
                        data[geo_and_time + disaggregations], 
                        how='inner', 
                        left_on=['countryCode'], 
                        right_on=['geoAreaCode']).reset_index()
                      
x = country_data.copy()
x.head()

,index,countryCode,countryName,LDC,LLDC,SIDS,Development,regionCode,regionName,geoAreaCode,geoAreaName,years,min_year,max_year,n_years,age_code,sex_code
0,0,8,Albania,NaN,NaN,NaN,Developed,513,Europe and Northern America,8,Albania,"[2002, 2005, 2008, 2012]",2002,2012,4,Y15T24,F
1,1,8,Albania,NaN,NaN,NaN,Developed,513,Europe and Northern America,8,Albania,"[2002, 2005, 2008, 2012]",2002,2012,4,Y_GE15,F
2,2,8,Albania,NaN,NaN,NaN,Developed,513,Europe and Northern America,8,Albania,"[2002, 2005, 2008, 2012]",2002,2012,4,Y_GE25,F
3,3,8,Albania,NaN,NaN,NaN,Developed,513,Europe and Northern America,8,Albania,"[2002, 2005, 2008, 2012]",2002,2012,4,Y15T24,M
4,4,8,Albania,NaN,NaN,NaN,Developed,513,Europe and Northern America,8,Albania,"[2002, 2005, 2008, 2012]",2002,2012,4,Y_GE15,M


#### Check number of years and latest year available, as well as the number of time series (disaggregations)

- `n_years` is the maximum number of years available for the series among any disaggregation
- `n_time_Series` is the number of disaggregations available for the series

In [108]:
country_availability = x.groupby(['countryCode', 'countryName'], as_index=False).agg({'max_year':['max'],'n_years':['max', 'count']})      
country_availability.columns = ['countryCode', 'countryName','max_year', 'n_years', 'n_timeSeries']

country_availability.head()

,countryCode,countryName,max_year,n_years,n_timeSeries
0,8,Albania,2012,4,9
1,12,Algeria,2011,1,9
2,24,Angola,2009,2,9
3,31,Azerbaijan,2005,4,9
4,32,Argentina,2017,17,9


#### Check the number of dimensions in the dataset

In [109]:
country_availability['n_dimensions']  = len(get_disaggregations(data)) 
country_availability.head()

,countryCode,countryName,max_year,n_years,n_timeSeries,n_dimensions
0,8,Albania,2012,4,9,2
1,12,Algeria,2011,1,9,2
2,24,Angola,2009,2,9,2
3,31,Azerbaijan,2005,4,9,2
4,32,Argentina,2017,17,9,2


#### Check average number levels/values per dimension

In [110]:
if len(get_disaggregations(data)) > 0:
    country_availability['values_per_dim']  =  country_availability['n_timeSeries'] ** (1/len(get_disaggregations(data)))  
else:
    country_availability['values_per_dim']  =  None

country_availability.head()

,countryCode,countryName,max_year,n_years,n_timeSeries,n_dimensions,values_per_dim
0,8,Albania,2012,4,9,2,3.0
1,12,Algeria,2011,1,9,2,3.0
2,24,Angola,2009,2,9,2,3.0
3,31,Azerbaijan,2005,4,9,2,3.0
4,32,Argentina,2017,17,9,2,3.0


#### Check disaggregation by sex

In [111]:
if 'sex_code' in get_disaggregations(data):
    xx = data[['geoAreaCode']][data['sex_code']=='F'].drop_duplicates()
    xx = xx['geoAreaCode'].tolist()
else:
    xx = []

country_availability['data_for_female'] = np.where(country_availability['countryCode'].isin(xx), 1, 0)

country_availability.head()

,countryCode,countryName,max_year,n_years,n_timeSeries,n_dimensions,values_per_dim,data_for_female
0,8,Albania,2012,4,9,2,3.0,1
1,12,Algeria,2011,1,9,2,3.0,1
2,24,Angola,2009,2,9,2,3.0,1
3,31,Azerbaijan,2005,4,9,2,3.0,1
4,32,Argentina,2017,17,9,2,3.0,1


#### Add regional groupings

In [112]:
                      
country_availability = pd.merge(countries,
                                country_availability,
                                how='left',
                                on = ['countryCode']
                                ).reset_index()

country_availability.head()

,index,countryCode,countryName_x,LDC,LLDC,SIDS,Development,regionCode,regionName,countryName_y,max_year,n_years,n_timeSeries,n_dimensions,values_per_dim,data_for_female
0,0,4,Afghanistan,x,x,NaN,Developing,62,Central and Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,8,Albania,NaN,NaN,NaN,Developed,513,Europe and Northern America,Albania,2012.0,4.0,9.0,2.0,3.0,1.0
2,2,12,Algeria,NaN,NaN,NaN,Developing,747,Northern Africa and Western Asia,Algeria,2011.0,1.0,9.0,2.0,3.0,1.0
3,3,20,Andorra,NaN,NaN,NaN,Developed,513,Europe and Northern America,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,24,Angola,x,NaN,NaN,Developing,202,Sub-Saharan Africa,Angola,2009.0,2.0,9.0,2.0,3.0,1.0


#### Add goal, target, indicator and series codes and descriptions

- Notice that goal and target codes are prefixed by "'", so they are taken as strings in excel

In [113]:
country_availability['Goal'] = '\''+ g['code']
country_availability['Target'] = '\'' + t['code']
country_availability['Indicator_ref'] = i['reference']
country_availability['Series_code'] = s['code']
country_availability['Series_desc'] = s['description']

country_availability.head()

,index,countryCode,countryName_x,LDC,LLDC,SIDS,Development,regionCode,regionName,countryName_y,...,n_years,n_timeSeries,n_dimensions,values_per_dim,data_for_female,Goal,Target,Indicator_ref,Series_code,Series_desc
0,0,4,Afghanistan,x,x,NaN,Developing,62,Central and Southern Asia,NaN,...,NaN,NaN,NaN,NaN,NaN,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...
1,1,8,Albania,NaN,NaN,NaN,Developed,513,Europe and Northern America,Albania,...,4.0,9.0,2.0,3.0,1.0,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...
2,2,12,Algeria,NaN,NaN,NaN,Developing,747,Northern Africa and Western Asia,Algeria,...,1.0,9.0,2.0,3.0,1.0,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...
3,3,20,Andorra,NaN,NaN,NaN,Developed,513,Europe and Northern America,NaN,...,NaN,NaN,NaN,NaN,NaN,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...
4,4,24,Angola,x,NaN,NaN,Developing,202,Sub-Saharan Africa,Angola,...,2.0,9.0,2.0,3.0,1.0,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...


### Availability by region

Regional availability is the aggregation of individual-country availablity. 
- Median of the number of years available per country
- Median of the number of time series avaialable per country
- Median of the number of dimensions available per country
- Median of the number of average number levels/values per dimension per country
- Mean of the % of observations for female


#### Calculate 'median/average' country availability by region

In [114]:
x = country_availability.copy()

if len(get_disaggregations(data)) > 0:
    country_availability_by_region = x.groupby(['Goal','Target','Indicator_ref','Series_code', 'Series_desc', 'regionCode', 'regionName'], 
                                                as_index=False).agg({'max_year':['count','median'],
                                                                    'n_years':['median'],
                                                                    'n_timeSeries':['median'],
                                                                    'n_dimensions':['median'],
                                                                    'values_per_dim':['median'],
                                                                    'data_for_female':['mean']})
else:
    country_availability_by_region = x.groupby(['Goal','Target','Indicator_ref','Series_code','Series_desc','regionCode', 'regionName'], 
                                                as_index=False).agg({'max_year':['count','median'],
                                                                    'n_years':['median'],
                                                                    'n_timeSeries':['median'],
                                                                    'n_dimensions':['median']})
        
    country_availability_by_region['values_per_dim'] = None
    country_availability_by_region['data_for_female'] = None

country_availability_by_region.columns = ['Goal','Target','Indicator_ref','Series_code','Series_desc','regionCode', 'regionName',
                                            'n_countries_available', 'median_max_year', 'median_n_years', 'median_n_timeSeries',
                                            'n_dimensions', 'median_values_per_dim', 'mean_data_for_female'
                                            ]

country_availability_by_region = pd.merge(country_availability_by_region, 
                                        x.groupby(['regionCode'],as_index=False)['countryCode'].agg('count'),
                                        how='left',
                                        on = ['regionCode']).reset_index()

country_availability_by_region.rename(columns={'countryCode': 'n_countries_total'}, inplace=True)

country_availability_by_region.head()

,index,Goal,Target,Indicator_ref,Series_code,Series_desc,regionCode,regionName,n_countries_available,median_max_year,median_n_years,median_n_timeSeries,n_dimensions,median_values_per_dim,mean_data_for_female,n_countries_total
0,0,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...,53,Australia and New Zealand,0,NaN,NaN,NaN,NaN,NaN,NaN,2
1,1,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...,62,Central and Southern Asia,12,2016.0,4.5,9.0,2.0,3.0,1.0,14
2,2,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...,202,Sub-Saharan Africa,40,2014.0,3.0,9.0,2.0,3.0,1.0,48
3,3,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...,419,Latin America and the Caribbean,20,2017.0,16.0,9.0,2.0,3.0,1.0,33
4,4,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...,513,Europe and Northern America,8,2015.0,13.0,9.0,2.0,3.0,1.0,45


#### Calculate overall average/median availability

In [115]:
x = country_availability.copy()

if len(get_disaggregations(data)) > 0:
    country_availability_world = x.groupby(['Goal','Target','Indicator_ref','Series_code','Series_desc'], 
                                            as_index=False).agg({'max_year':['count','median'],
                                                                'n_years':['median'],
                                                                'n_timeSeries':['median'],
                                                                'n_dimensions':['median'],
                                                                'values_per_dim':['median'],
                                                                'data_for_female':['mean']})
    
else:
    country_availability_world = x.groupby(['Goal','Target','Indicator_ref','Series_code', 'Series_desc'], 
                                            as_index=False).agg({'max_year':['count','median'],
                                                                'n_years':['median'],
                                                                'n_timeSeries':['median'],
                                                                'n_dimensions':['median']}) 
    country_availability_world['values_per_dim'] = None
    country_availability_world['data_for_female'] = None

country_availability_world.columns = ['Goal','Target','Indicator_ref','Series_code','Series_desc',
                                            'n_countries_available', 'median_max_year', 'median_n_years', 'median_n_timeSeries',
                                            'n_dimensions', 'median_values_per_dim', 'mean_data_for_female'
                                            ]
        
        
country_availability_world = pd.merge(country_availability_world, 
                                        x.groupby(['Series_code','Series_desc'],as_index=False)['countryCode'].agg('count'),
                                        how='left',
                                        on = ['Series_code','Series_desc']).reset_index()

country_availability_world.rename(columns={'countryCode': 'n_countries_total'}, inplace=True)

country_availability_world['regionCode'] = 1
country_availability_world['regionName'] = 'World'

country_availability_world.head()

,index,Goal,Target,Indicator_ref,Series_code,Series_desc,n_countries_available,median_max_year,median_n_years,median_n_timeSeries,n_dimensions,median_values_per_dim,mean_data_for_female,n_countries_total,regionCode,regionName
0,0,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...,110,2015.0,4.0,9.0,2.0,3.0,1.0,194,1,World


#### Write to file mean/average country availability for each regional group and for the world 

In [116]:
frames = [country_availability_world,country_availability_by_region ]
availability_by_series_and_region = pd.concat(frames)
availability_by_series_and_region.shape
availability_by_series_and_region

(9, 16)

,index,Goal,Target,Indicator_ref,Series_code,Series_desc,n_countries_available,median_max_year,median_n_years,median_n_timeSeries,n_dimensions,median_values_per_dim,mean_data_for_female,n_countries_total,regionCode,regionName
0,0,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...,110,2015.0,4.0,9.0,2.0,3.0,1.0,194,1,World
0,0,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...,0,NaN,NaN,NaN,NaN,NaN,NaN,2,53,Australia and New Zealand
1,1,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...,12,2016.0,4.5,9.0,2.0,3.0,1.0,14,62,Central and Southern Asia
2,2,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...,40,2014.0,3.0,9.0,2.0,3.0,1.0,48,202,Sub-Saharan Africa
3,3,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...,20,2017.0,16.0,9.0,2.0,3.0,1.0,33,419,Latin America and the Caribbean
4,4,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...,8,2015.0,13.0,9.0,2.0,3.0,1.0,45,513,Europe and Northern America
5,5,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...,3,2013.0,2.0,9.0,2.0,3.0,1.0,12,543,Oceania (excluding Australia and New Zealand)
6,6,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...,15,2014.0,4.0,9.0,2.0,3.0,1.0,24,747,Northern Africa and Western Asia
7,7,'1,'1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...,12,2015.0,6.0,9.0,2.0,3.0,1.0,16,753,Eastern and South-Eastern Asia


In [ ]:
f

### Availability by indicator

In [ ]:
Availability_by_indicator = availability_by_series_and_region

x.median_values_per_dim = pd.to_numeric(x.median_values_per_dim)
x.mean_data_for_female= pd.to_numeric(x.mean_data_for_female)

availability_by_indicator_and_region = x.groupby(['Goal','Target','Indicator_ref', 'regionCode', 'regionName'], 
                                                  as_index=False).agg({'n_countries_available':['mean'],
                                                                      'median_max_year':['mean'],
                                                                      'median_n_years':['mean'],
                                                                      'median_n_timeSeries':['mean'],
                                                                      'n_dimensions':['mean'],
                                                                      'median_values_per_dim':['mean'],
                                                                      'mean_data_for_female':['mean'],
                                                                      'n_countries_total':['mean']})

availability_by_indicator_and_region.columns = ['Goal',
'Target',
'Indicator_ref',
'regionCode',
'regionName',
'avg_n_countries_available',
'avg_median_max_year',
'avg_median_n_years',
'avg_median_n_timeSeries',
'avg_n_dimensions',
'avg_median_values_per_dim',
'mean_data_for_female',
'n_countries_total']


                     
       

In [ ]:
x = availability_by_indicator_and_region.copy()

x.median_values_per_dim = pd.to_numeric(x.avg_median_values_per_dim)
x.mean_data_for_female= pd.to_numeric(x.mean_data_for_female)

availability_by_target_and_region = x.groupby(['Goal','Target', 'regionCode', 'regionName'], 
                                                  as_index=False).agg({'avg_n_countries_available':['mean'],
                                                                      'avg_median_max_year':['mean'],
                                                                      'avg_median_n_years':['mean'],
                                                                      'avg_median_n_timeSeries':['mean'],
                                                                      'avg_n_dimensions':['mean'],
                                                                      'avg_median_values_per_dim':['mean'],
                                                                      'mean_data_for_female':['mean'],
                                                                      'n_countries_total':['mean']})


availability_by_target_and_region.columns = ['Goal',
'Target',
'regionCode',
'regionName',
'avg_n_countries_available',
'avg_median_max_year',
'avg_median_n_years',
'avg_median_n_timeSeries',
'avg_n_dimensions',
'avg_median_values_per_dim',
'mean_data_for_female',
'n_countries_total']


                    

In [ ]:
x = availability_by_target_and_region.copy()

x.median_values_per_dim = pd.to_numeric(x.avg_median_values_per_dim)
x.mean_data_for_female= pd.to_numeric(x.mean_data_for_female)

availability_by_goal_and_region = x.groupby(['Goal','regionCode', 'regionName'], 
                                                  as_index=False).agg({'avg_n_countries_available':['mean'],
                                                                      'avg_median_max_year':['mean'],
                                                                      'avg_median_n_years':['mean'],
                                                                      'avg_median_n_timeSeries':['mean'],
                                                                      'avg_n_dimensions':['mean'],
                                                                      'avg_median_values_per_dim':['mean'],
                                                                      'mean_data_for_female':['mean'],
                                                                      'n_countries_total':['mean']})


availability_by_goal_and_region.columns = ['Goal',
'regionCode',
'regionName',
'avg_n_countries_available',
'avg_median_max_year',
'avg_median_n_years',
'avg_median_n_timeSeries',
'avg_n_dimensions',
'avg_median_values_per_dim',
'mean_data_for_female',
'n_countries_total']


                    